<a href="https://colab.research.google.com/github/dhruvagja/DBMS-term-project/blob/main/DBMS_term_project_Dextop_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://uafys8te3y-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
%%capture
from flask import Flask, render_template, request
import torch
from transformers import AutoTokenizer
import psycopg2
from tabulate import tabulate
import re

!pip install peft

### Creating a postgresql account to run in Colab

In [ ]:
%%capture
!sudo apt update
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

# !sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo -u postgres psql -U postgres -c "CREATE ROLE root WITH SUPERUSER;"
!sudo -u postgres psql -U postgres -c "ALTER ROLE root WITH LOGIN;"
!sudo -u postgres psql -U postgres -c "CREATE ROLE postgres WITH PASSWORD 'postgres';"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"


ERROR:  role "root" already exists
ALTER ROLE
ERROR:  role "postgres" already exists
ALTER ROLE


In [ ]:

app = Flask(__name__, template_folder='/content/drive/MyDrive/DBMS-term-project/templates')

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = torch.load('/content/drive/MyDrive/DBMS-term-project/model/text2sql.pt')
# model = torch.load('/content/drive/MyDrive/DBMS-term-project/model/text2sql.pt', map_location=torch.device('cpu'))

def chat_template(question, context):

    template = f"""\
    <|im_start|>user
    Given the context, generate an SQL query for the following question
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

@app.route('/', methods=['GET', 'POST'])
def main():
    # return render_template('main.html')
     if request.method == 'GET':
        return(render_template('main.html'))

     if request.method == 'POST':
        question = request.form.get("question")
        predict_text = "Question: "
        table = request.form.get("context")
        def generate_create_table_statements(input_string):
            table_definitions = [table.strip() for table in input_string.split(':')]

            create_table_statements = []
            for table_definition in table_definitions:
                table_parts = table_definition.split('(', maxsplit=1)
                if len(table_parts) == 2:  # Ensure there are both table name and columns
                    table_name = table_parts[0].strip()
                    columns = '(' + table_parts[1]  # Add parentheses back to the columns
                    create_table_statement = f"CREATE TABLE {table_name} {columns};"
                    create_table_statements.append(create_table_statement)
                else:
                    print(f"Invalid table definition: {table_definition}")

            return create_table_statements

        contexts = generate_create_table_statements(table)
        context = ""
        for c in contexts:
            context = context + c + " "
        prompt = chat_template(question, context)
        print(prompt)

        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        # inputs = tokenizer(prompt, return_tensors="pt")
        output = model.generate(**inputs, max_new_tokens=512)
        text = tokenizer.decode(output[0], skip_special_tokens=True)
        sql_query = re.search(r'assistant\n(.*?)(\n|$)', text, re.DOTALL).group(1)
        print(sql_query)

        sql_query = sql_query + ";"
        sql_query = sql_query.replace('"', "'")
        error_message=""
        try:
            cursor.execute(sql_query)
        except Exception as e:
            print("Query failed to execute.", e)
            error_message="Failed to generate query, Rewrite the question and context"
            return(render_template('main.html',predict_text=predict_text, question=question, query=sql_query,result="Query failed", error_message=error_message))
        rows = cursor.fetchall()

        print("\nQuery output: ")
        for row in rows:
          print(row)

        query_text = "Query genereated: "

        return(render_template('main.html', predict_text=predict_text, question=question, query=sql_query, query_text=query_text, result=tabulate(rows, tablefmt="psql")))

if __name__=="__main__":
    try:
        connection = psycopg2.connect(
            host="localhost",
            port=5432,
            database="postgres",
            user="postgres",
            password="postgres"
        )
        cursor = connection.cursor()
        print("Connection established successfully!")
        with open('/content/drive/MyDrive/DBMS-term-project/new_db.txt', 'r') as file:
            text = file.read()
        cursor.execute(text)

    except Exception as e:
        print("Error connecting to the database:", e)



    app.run()

Connection established successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


### Hosting app on public host